# YOLO Step 3 v.3 - with Synthetic data

Roboflow data generated:
2022-03-09 7:16pm
Version 3
Generated Mar 9, 2022

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import io
import pandas as pd
import json
import os
import glob 


In [2]:
# change directory and copy files locally -- only do once
# Roboflow output

!rm -r /home/ec2-user/AutoArki/yolov5/yolov5/training_data
!mkdir /home/ec2-user/AutoArki/yolov5/yolov5/training_data
!cp /home/ec2-user/train/images/*  /home/ec2-user/AutoArki/yolov5/yolov5/training_data --recursive
!cp /home/ec2-user/train/labels/*  /home/ec2-user/AutoArki/yolov5/yolov5/training_data --recursive

!rm -r /home/ec2-user/AutoArki/yolov5/yolov5/validation_data
!mkdir /home/ec2-user/AutoArki/yolov5/yolov5/validation_data
!cp /home/ec2-user/valid/images/*  /home/ec2-user/AutoArki/yolov5/yolov5/validation_data --recursive
!cp /home/ec2-user/valid/labels/*  /home/ec2-user/AutoArki/yolov5/yolov5/validation_data --recursive

!rm -r /home/ec2-user/AutoArki/yolov5/yolov5/test_data
!mkdir /home/ec2-user/AutoArki/yolov5/yolov5/test_data
!cp /home/ec2-user/test/images/* /home/ec2-user/AutoArki/yolov5/yolov5/test_data --recursive
!cp /home/ec2-user/test/labels/* /home/ec2-user/AutoArki/yolov5/yolov5/test_data --recursive


rm: cannot remove ‘/home/ec2-user/AutoArki/yolov5/yolov5/validation_data’: No such file or directory


# Check 

In [2]:
def count_classes(file_path):
    '''
    increment class_counts dictionary for each instance of a 
    class check against roboflow summary (health check)
    
    '''
    infile = open(file_path, "r")
    lines = infile.readlines()

    if len(lines) == 0:
        pass
    else:
        for og_line in lines:
            class_counts[og_line[0]] +=1
    return class_counts

In [3]:
#Count of class types in train
class_counts = {'0':0, '1':0, '2':0, '3': 0, '4': 0, '5': 0, '6':0}

os.chdir("/home/ec2-user/AutoArki/yolov5/yolov5/training_data")
for file in glob.glob("*.txt"):
    count_classes(file)
print(class_counts)

{'0': 13, '1': 572, '2': 78, '3': 6882, '4': 861, '5': 887, '6': 0}


# Replace Class Type

In [4]:
classes = {'1': '0', #Column_Wall_Intersection
           '3': '1', #illegibletext'
           '4': '2', #'no_stair_direction'
           '5': '3'  #missing_wall
          } 


In [5]:
def replace_class_n_with_class_name(file_path):
    '''for a txt file replace the class number with a class name'''
    new_lines = []
    infile = open(file_path, "r")
    lines = infile.readlines()

    if len(lines) == 0:
        pass
    else:
        for og_line in lines:
            class_n = og_line[0]
            if class_n == '0' or class_n == '2': #bad annotation needs to be removed and CMU wall hinge insufficient data
                continue
            class_str = classes[class_n]
            new_line = og_line.replace(class_n, class_str, 1)
            new_lines.append(new_line)

    with open(file_path, 'w') as outfile:
        outfile.write(''.join(new_lines))

In [7]:
# replace class names for test and validation txt files

os.chdir("/home/ec2-user/AutoArki/yolov5/yolov5/training_data")
for file in glob.glob("*.txt"):
    replace_class_n_with_class_name(file)
    
os.chdir("/home/ec2-user/AutoArki/yolov5/yolov5/validation_data")
for file in glob.glob("*.txt"):
    replace_class_n_with_class_name(file)    
    

# Learning

use smallest YOLO model to start yolov5s.pt

In [8]:
import torch
import torchvision

In [9]:
os.chdir("/home/ec2-user/AutoArki/yolov5/yolov5/")
#!pip install -r requirements.txt # first run
!python train.py\
    --data custom_dataset.yaml \
    --epochs 75\
    --project custom_yolov5 \
    --bbox_interval 1 \
    --save-period 5 \
    --weights yolov5s.pt 

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=yolov5s.pt, cfg=, data=custom_dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=75, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=custom_yolov5, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=5, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=1, artifact_alias=latest
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (6/6), done.
remote: Total 11 (delta 6), reused 6 (delta 6), pack-reused 5
Unpacking objects: 100% (11/11), 4.73 KiB | 1.18 MiB/s, done.
From https://github.co

No stair direction Class 2 is yielding low results. I'm guessing this has to do with tight bounds on this images. Because arrows or chinese characters nearby can indicate direction. 

In [ ]:
# !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="pvpFbwxA6FFS6pAwYdNk")
# project = rf.workspace("new-workspace-s32a1").project("autoarki")
# dataset = project.version(4).download("yolov5")